In [1]:
import numpy as np
import matplotlib.pyplot as plt
from gastimator import gastimator
import pandas as pd
from gastimator import priors
from gastimator import corner_plot
import math

In [2]:
SED=pd.read_csv('Documents/NuclearSED.csv')
mcmcdet=pd.read_csv('Documents/MCMCfit.csv')
SEDdust=pd.read_csv('Documents/DustSED.csv')
mcmcdust=pd.read_csv('Documents/MCMCdust.csv')

In [3]:
def mod_black_body(values,v,D,T,b):
    d=D*3.0856e22
    mlog=10**(values)
    m=mlog*2e30
    c=3e8
    h=6.626e-34
    kb=1.38e-23
    v0=6e11
    B=((2*h*v**3/c**2)*1/(np.exp(h*v/(kb*T))))
    k0=0.051
    k=k0*((v/v0)**b)
    S=(k*B*m/(d**2))
    sj=S/1e-26
    return sj

In [4]:
def power_law(values,x):
    B=(values[1]*(x/1e10)**values[0])
    return B

In [5]:
def combined(values,x,D,T,b):
    com=power_law(values[0:2],x)+mod_black_body(values[2],x,D,T,b)
    return com

In [6]:
i=22
SEDfreq=mcmcdet['SED Freq']
SEDflux=mcmcdet['SED Flux']
SEDRMS=mcmcdet['SED RMS']
xfreq=SED[SEDfreq[i]]
datafl=SED[SEDflux[i]]
rms=SED[SEDRMS[i]]
rmstot=np.sqrt(((rms/datafl)*100)**2+10**2)
#xfreq=[]
#datafl=[]
#rms=[]
usb=mcmcdet['USB Flux']
lsb=mcmcdet['LSB Flux']
usbfreq=mcmcdet['USB Freq']
lsbfreq=mcmcdet['LSB Freq']
usbrms=mcmcdet['USB RMS']
lsbrms=mcmcdet['LSB RMS']
alma=np.array([lsb[i],usb[i]])
almafreq=np.array([lsbfreq[i],usbfreq[i]])
almarms=np.array([lsbrms[i],usbrms[i]])
xfreq=np.append(xfreq,almafreq)
datafl=np.append(datafl,alma)
rms=np.append(rms,almarms)
temp=mcmcdet['Temperature']
beta=mcmcdet['Beta']
distance=mcmcdet['Distance']
indexmax=mcmcdet['Index Max']
indexmin=mcmcdet['Index Min']
indexguess=mcmcdet['Index Guess']
interceptmax=mcmcdet['Intercept Max']
interceptmin=mcmcdet['Intercept Min']
interceptguess=mcmcdet['Intercept Guess']
massmax=mcmcdet['Mass Max']
massmin=mcmcdet['Mass Min']
massguess=mcmcdet['Mass Guess']
mmax=np.log10(massmax)
mguess=np.log10(massguess)
c=3e5
H0=70
z=(distance[i]/c)*H0
xfreqe=(1+z)*xfreq

In [133]:
mcmccom=gastimator(combined,xfreqe,distance[i],temp[i],beta[i])

In [134]:
mcmccom.labels=np.array(['Index','Intercept','Mass'])
mcmccom.guesses=np.array([indexguess[i],interceptguess[i],mguess[i]])
mcmccom.min=np.array([indexmin[i],interceptmin[i],massmin[i]])
mcmccom.max=np.array([indexmax[i],interceptmax[i],mmax[i]])
mcmccom.fixed=np.array([False,False,False])
mcmccom.precision=np.array([1.,1.,1e2])

In [135]:
nsamplescom=10000
outputvaluecom,outputllcom=mcmccom.run(datafl,rms,nsamplescom,nchains=1,plot=False)

Doing chain 1
     Chain has not converged - Accept rate: 0.13333333333333333
     Chain has not converged - Accept rate: 0.17333333333333334
     Target rate not reached
Chain converged: LL: -1.2079673723964097 - Accept rate:0.5266666666666666
Best chain so far!
Best fit:
  Index: -0.4904596334172557
  Intercept: 0.014180361308677978
  Mass: 5.389590354435199
Starting final chain
Final best fit values and 1sigma errors:
  Index: -0.48658845098084863 ± 0.006328766231702115
  Intercept: 0.014314494970396233 ± 0.0002270384000146843
  Mass: 3.578366655134165 + 1.3382514320709613 - 1.6597719653782224


In [176]:
np.savetxt('7172.csv',outputvaluecom,delimiter=',')
np.savetxt('7172ll.csv',outputllcom,delimiter=',')